# GFK

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#adaptacion
from utils.adaptacion import *

#clasificadores
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from utils.clasificacion import *

#carga de datasets
from utils.DatasetStorage import Dataset
from utils.paths import *

#otros
import os
import numpy as np
import pandas as pd
from sklearn.externals import joblib

#variables para guardar los resultados
tipo = pruebas[2]
dataset_name = datasets[0]
dims = dimensions[dataset_name]

Using Theano backend.
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


## Pruebas con el dataset Amazon

In [3]:
print tipo
print dataset_name
print dims
print data_path

gfk
amazon
3000
data


In [4]:
# cargando dataset Amazon
dataset_path = os.path.join(data_path, dataset_name+'.pkl')
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
unlabeled = dataset_object.unlabeled
domains = dataset_object.domains

Dataset already splitted


### Creación de modelos de adaptación.

Aumentar la cantidad de sub-espacios de 10 a 50 mejora los resultados, mientras que aumentar la dimensionalidad de los sub-espacios empeora o mantiene los resultados obtenidos.

En este dataset (con 2000 dimensiones) se encuentra que fijando a los sub-espacios con dimensionalidad 500 o 1000 muestra los mismos resultados, por lo que solamente se realizan pruebas con 50 sub-espacios de dimensionalidad 750.

In [ ]:
#por cada par se entrena un adaptador y se guarda el que mejor resultados entrega

size_dims = [int(dims/4)]
n_subs = [50]

parameters = {
    'dims': size_dims,
    'n_subs': n_subs
}


print "Adaptando entre:"
for src in domains:
    for tgt in domains:
        if src is not tgt:
            print "%s - %s" % (src, tgt)
            X_tr = np.asarray(labeled[src]['X_tr'].todense())
            y_tr = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)

            X_ts = np.asarray(labeled[tgt]['X_ts'].todense())
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            gfk, score = gfk_grid_search(parameters, X_tr, y_tr, X_ts, n_jobs=4)
            print "\tMejor modelo: %.3f" % score
            
            
            gfk_save_path = os.path.join(models_path, tipo, "%s_%s_%s.pkl" % (dataset_name, src,tgt))
            print "\tGuardando modelo en %s" % gfk_save_path
            joblib.dump(gfk, gfk_save_path)

print "\nAdaptaciones completadas"

### Pruebas de clasificación

In [5]:
df = pd.DataFrame(columns=dataframe_columns)

i=0
# por cada par posible para adaptar
for src in domains:
    for tgt in domains:
        if src is not tgt:
            print "Tarea %d de 12" % (i+1)
            
            
            #baseline in-domain error
            #e_b(T,T)
            #entrenado en dominio tgt y probado en dominio tgt
            X_tr = np.asarray(labeled[tgt]['X_tr'].todense())
            y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)
            
            X_ts = np.asarray(labeled[tgt]['X_ts'].todense())
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            # se crean las rutas para cargar o crear los modelos
            model_name = "indomain_%s.pkl" % (tgt)
            model_path = os.path.join(models_path, dataset_name, model_name)
            
            #Se realiza una clasificacion, estimando los parametros mediante cv
            svc = load_best_score(model_path, X_tr, y_tr)
            b_error = 1-svc.score(X_ts, y_ts)
            
            #############
            #### GFK ####
            #############
            # se adaptan los dominios usando GFK
            print "Adaptando dominios..."
            best_model_path = os.path.join(models_path, tipo, "%s_%s_%s.pkl" % (dataset_name, src, tgt))
            gfk = joblib.load(best_model_path)
            
            
            X_tr = np.asarray(labeled[src]['X_tr'].todense())
            X_tr_a = transform_gfk(X_tr, gfk)
            y_tr = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)
            
            X_ts = np.asarray(labeled[tgt]['X_ts'].todense())
            X_ts_a = transform_gfk(X_ts, gfk)
            y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)
            
            clf = get_best_score(X_tr_a, y_tr, clasifier='SVC', n_jobs=4)
            t_error = 1-clf.score(X_ts_a, y_ts)
            #t_error = gfk_compute_accuracy(gfk.G, X_tr, y_tr, X_ts, y_ts)
            
            
            # transfer loss t
            # t_error - b_error
            t_loss = t_error - b_error
            
            tarea = src[0]+'->'+tgt[0]
            df.loc[i] = ['GFK',tarea,src,tgt,b_error*100,t_error*100, t_loss*100]
            
            i += 1
            
print "\nPruebas completadas."

Tarea 1 de 12
Cargando modelo existente.
Adaptando dominios...


/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with Standar

Tarea 2 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 3 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 4 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 5 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 6 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 7 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 8 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 9 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 10 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 11 de 12
Cargando modelo existente.
Adaptando dominios...
Tarea 12 de 12
Cargando modelo existente.
Adaptando dominios...

Pruebas completadas.


In [6]:
df

,Adaptacion,Tarea,Fuente,Objetivo,Baseline error,Transfer error,Transfer loss
0,GFK,e->d,electronics,dvd,23.00,27.50,4.50
1,GFK,e->k,electronics,kitchen,17.50,19.75,2.25
2,GFK,e->b,electronics,books,21.25,35.00,13.75
3,GFK,d->e,dvd,electronics,20.00,30.25,10.25
4,GFK,d->k,dvd,kitchen,17.50,26.00,8.50
5,GFK,d->b,dvd,books,21.25,32.25,11.00
6,GFK,k->e,kitchen,electronics,20.00,18.25,-1.75
7,GFK,k->d,kitchen,dvd,23.00,30.50,7.50
8,GFK,k->b,kitchen,books,21.25,35.75,14.50
9,GFK,b->e,books,electronics,20.00,25.25,5.25


In [7]:
new_scores_path = os.path.join(scores_path, tipo, dataset_name+".csv")

print "Guardando en %s" % new_scores_path
df.to_csv(new_scores_path, columns=df.columns)
print "Resultados guardados."

Guardando en scores/gfk/amazon.csv
Resultados guardados.
